# Tugas Skema Masalah - Kecerdasan Buatan

## Member of Group

- Federico Matthew Pratama - 233405001
- Fernando Perry - 233406005

### Diskusi dengan Assisten Pribadi

[GitHub Copilot](https://github.com/copilot/share/c269410c-0044-8871-a002-aa46a0da6863)

### Visualisasi Lengkap (Google Spreadsheet)

[Google Spreadsheet](https://docs.google.com/spreadsheets/d/1xClQWvudqFlrtaXHDu6QwPMDLT7qxlPU4y8j58fPvqk/edit?usp=sharing)

### Persiapan Data awal (.csv) dengan nama `data1.csv`

In [ ]:
data = """Tautan,Bobot,Tau,Eta
A-B,4,1,0.25
A-C,3,1,0.333333
A-D,6,1,0.166667
B-C,2,1,0.5
B-D,5,1,0.2
C-D,1,1,1.0"""
with open('data1.csv', 'w') as the_file:
    the_file.write(data)

### Menentukan Probabilitas

In [ ]:
import pandas as pd
import numpy as np
import random

def read_graph_data(filename):
    """Read graph data from a CSV file."""
    try:
        df = pd.read_csv(filename)
        print("Data loaded successfully:")
        print(df)
        return df
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None

def create_graph(df):
    """Create a graph representation from the dataframe."""
    # Extract unique nodes
    nodes = set()
    for route in df['Tautan']:
        nodes.add(route.split('-')[0])
        nodes.add(route.split('-')[1])
    nodes = sorted(list(nodes))

    # Create distance and pheromone matrices
    n = len(nodes)
    distances = np.zeros((n, n))
    pheromones = np.zeros((n, n))

    # Map node names to indices
    node_indices = {node: i for i, node in enumerate(nodes)}

    # Fill the matrices
    for _, row in df.iterrows():
        start, end = row['Tautan'].split('-')
        i, j = node_indices[start], node_indices[end]
        distances[i, j] = row['Bobot']
        distances[j, i] = row['Bobot']  # Assuming an undirected graph
        pheromones[i, j] = row['Tau']
        pheromones[j, i] = row['Tau']  # Assuming an undirected graph

    return nodes, node_indices, distances, pheromones

def calculate_probabilities(distances, pheromones, alpha=1, beta=1):
    """Calculate transition probabilities for each edge."""
    n = len(distances)
    probabilities = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if distances[i, j] > 0:  # If there is an edge
                # Calculate Eta (η = 1/distance)
                eta = 1.0 / distances[i, j]
                # Calculate probability numerator (τ^α * η^β)
                probabilities[i, j] = (pheromones[i, j] ** alpha) * (eta ** beta)

    # Normalize probabilities for each row
    for i in range(n):
        row_sum = np.sum(probabilities[i, :])
        if row_sum > 0:
            probabilities[i, :] = probabilities[i, :] / row_sum

    return probabilities

def ant_colony_optimization(nodes, node_indices, distances, pheromones,
                           alpha=1, beta=1, num_ants=10, num_iterations=100,
                           evaporation_rate=0.5, Q=100):
    """Main ACO algorithm."""
    n = len(nodes)
    best_path = None
    best_path_length = float('inf')

    # Store history for visualization
    best_path_history = []

    # Main loop
    for iteration in range(num_iterations):
        paths = []
        path_lengths = []

        # For each ant
        for ant in range(num_ants):
            # Choose a random starting node
            current_node = random.randint(0, n-1)
            path = [current_node]
            visited = [False] * n
            visited[current_node] = True
            path_length = 0

            # Construct a solution
            for _ in range(n-1):
                # Calculate transition probabilities
                probs = np.zeros(n)
                for j in range(n):
                    if not visited[j] and distances[current_node, j] > 0:
                        # Calculate Eta (η = 1/distance)
                        eta = 1.0 / distances[current_node, j]
                        # Calculate probability numerator (τ^α * η^β)
                        probs[j] = (pheromones[current_node, j] ** alpha) * (eta ** beta)

                # Normalize probabilities
                if np.sum(probs) > 0:
                    probs = probs / np.sum(probs)

                # Choose next node
                next_node = np.random.choice(n, p=probs)
                path.append(next_node)
                visited[next_node] = True
                path_length += distances[current_node, next_node]
                current_node = next_node

            # Add the distance to return to the starting point (if applicable)
            if distances[path[-1], path[0]] > 0:
                path_length += distances[path[-1], path[0]]

            paths.append(path)
            path_lengths.append(path_length)

            # Update best path
            if path_length < best_path_length:
                best_path = path
                best_path_length = path_length

        best_path_history.append(best_path_length)

        # Pheromone update
        # Evaporation
        pheromones = (1 - evaporation_rate) * pheromones

        # Reinforcement
        for ant in range(num_ants):
            path = paths[ant]
            path_length = path_lengths[ant]
            for i in range(len(path)-1):
                pheromones[path[i], path[i+1]] += Q / path_length
                pheromones[path[i+1], path[i]] += Q / path_length  # Assuming an undirected graph

        print(f"Iteration {iteration+1}: Best path length = {best_path_length}")

    # Convert node indices to names for the best path
    best_path_names = [nodes[i] for i in best_path]
    return best_path_names, best_path_length, best_path_history

def save_probabilities_to_csv(nodes, probabilities, filename="probabilities.csv"):
    """Save the probability matrix to a CSV file."""
    df = pd.DataFrame(probabilities, index=nodes, columns=nodes)
    df.to_csv(filename)
    print(f"Probabilities saved to {filename}")

def main():
    # Read data from CSV
    df = read_graph_data("data1.csv")
    if df is None:
        return

    # Create graph representation
    nodes, node_indices, distances, pheromones = create_graph(df)

    # Calculate and display initial probabilities
    probabilities = calculate_probabilities(distances, pheromones)
    print("\nInitial Probabilities:")
    for i, start_node in enumerate(nodes):
        for j, end_node in enumerate(nodes):
            if distances[i, j] > 0:
                print(f"{start_node}-{end_node}: {probabilities[i, j]:.6f}")

    # Save probabilities to CSV
    save_probabilities_to_csv(nodes, probabilities)

    # Run ACO algorithm
    print("\nRunning Ant Colony Optimization...")
    best_path, best_path_length, best_path_history = ant_colony_optimization(
        nodes, node_indices, distances, pheromones)

    print(f"\nBest path: {' -> '.join(best_path)}")
    print(f"Best path length: {best_path_length}")

if __name__ == "__main__":
    main()

Data loaded successfully:
  Tautan  Bobot  Tau       Eta
0    A-B      4    1  0.250000
1    A-C      3    1  0.333333
2    A-D      6    1  0.166667
3    B-C      2    1  0.500000
4    B-D      5    1  0.200000
5    C-D      1    1  1.000000

Initial Probabilities:
A-B: 0.333333
A-C: 0.444444
A-D: 0.222222
B-A: 0.263158
B-C: 0.526316
B-D: 0.210526
C-A: 0.181818
C-B: 0.272727
C-D: 0.545455
D-A: 0.121951
D-B: 0.146341
D-C: 0.731707
Probabilities saved to probabilities.csv

Running Ant Colony Optimization...
Iteration 1: Best path length = 13.0
Iteration 2: Best path length = 13.0
Iteration 3: Best path length = 13.0
Iteration 4: Best path length = 13.0
Iteration 5: Best path length = 13.0
Iteration 6: Best path length = 13.0
Iteration 7: Best path length = 13.0
Iteration 8: Best path length = 13.0
Iteration 9: Best path length = 13.0
Iteration 10: Best path length = 13.0
Iteration 11: Best path length = 13.0
Iteration 12: Best path length = 13.0
Iteration 13: Best path length = 13.0
Ite

### Update Feromon

In [ ]:
import pandas as pd
import numpy as np
import random

def read_graph_data(filename):
    """Read graph data from a CSV file."""
    try:
        df = pd.read_csv(filename)
        print("Data loaded successfully:")
        print(df)
        return df
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None

def create_graph(df):
    """Create a graph representation from the dataframe."""
    # Extract unique nodes
    nodes = set()
    for route in df['Tautan']:
        nodes.add(route.split('-')[0])
        nodes.add(route.split('-')[1])
    nodes = sorted(list(nodes))

    # Create distance and pheromone matrices
    n = len(nodes)
    distances = np.zeros((n, n))
    pheromones = np.zeros((n, n))

    # Map node names to indices
    node_indices = {node: i for i, node in enumerate(nodes)}

    # Fill the matrices
    for _, row in df.iterrows():
        start, end = row['Tautan'].split('-')
        i, j = node_indices[start], node_indices[end]
        distances[i, j] = row['Bobot']
        distances[j, i] = row['Bobot']  # Assuming an undirected graph
        pheromones[i, j] = row['Tau']
        pheromones[j, i] = row['Tau']  # Assuming an undirected graph

    return nodes, node_indices, distances, pheromones

def calculate_probabilities(distances, pheromones, alpha=1, beta=1):
    """Calculate transition probabilities for each edge."""
    n = len(distances)
    probabilities = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if distances[i, j] > 0:  # If there is an edge
                # Calculate Eta (η = 1/distance)
                eta = 1.0 / distances[i, j]
                # Calculate probability numerator (τ^α * η^β)
                probabilities[i, j] = (pheromones[i, j] ** alpha) * (eta ** beta)

    # Normalize probabilities for each row
    for i in range(n):
        row_sum = np.sum(probabilities[i, :])
        if row_sum > 0:
            probabilities[i, :] = probabilities[i, :] / row_sum

    return probabilities

def run_single_ant(nodes, node_indices, distances, pheromones, alpha=1, beta=1):
    """Simulate a single ant's journey and return the path and its length."""
    n = len(nodes)

    # Choose a random starting node
    current_node = random.randint(0, n-1)
    path = [current_node]
    visited = [False] * n
    visited[current_node] = True
    path_length = 0

    # Construct a solution
    for _ in range(n-1):
        # Calculate transition probabilities
        probs = np.zeros(n)
        for j in range(n):
            if not visited[j] and distances[current_node, j] > 0:
                # Calculate Eta (η = 1/distance)
                eta = 1.0 / distances[current_node, j]
                # Calculate probability numerator (τ^α * η^β)
                probs[j] = (pheromones[current_node, j] ** alpha) * (eta ** beta)

        # Normalize probabilities
        if np.sum(probs) > 0:
            probs = probs / np.sum(probs)

        # Choose next node
        next_node = np.random.choice(n, p=probs)
        path.append(next_node)
        visited[next_node] = True
        path_length += distances[current_node, next_node]
        current_node = next_node

    # Complete the tour by returning to the starting point (if applicable)
    if distances[path[-1], path[0]] > 0:
        path_length += distances[path[-1], path[0]]
        path.append(path[0])  # Complete the cycle

    return path, path_length

def calculate_delta_tau(df, nodes, node_indices, path, path_length, Q=100):
    """Calculate delta tau (pheromone deposit) for a given path."""
    n = len(nodes)
    delta_tau = np.zeros((n, n))

    # For each edge in the path
    for i in range(len(path)-1):
        start_node = path[i]
        end_node = path[i+1]
        delta_tau[start_node, end_node] = Q / path_length
        delta_tau[end_node, start_node] = Q / path_length  # For undirected graph

    # Create a dataframe to store the results
    results = []
    for _, row in df.iterrows():
        start, end = row['Tautan'].split('-')
        i, j = node_indices[start], node_indices[end]
        delta = delta_tau[i, j]
        results.append({
            'Tautan': row['Tautan'],
            'Bobot': row['Bobot'],
            'Tau': row['Tau'],
            'Eta': 1.0 / row['Bobot'],
            'Delta Tau': delta,
            'Tau Baru': (1 - rho) * row['Tau'] + delta
        })

    return pd.DataFrame(results)

def main():
    # Parameters
    global rho  # Evaporation rate
    rho = 0.5
    Q = 100  # Pheromone deposit constant

    # Read data from CSV
    df = read_graph_data("data1.csv")
    if df is None:
        return

    # Create graph representation
    nodes, node_indices, distances, pheromones = create_graph(df)

    # Calculate and display initial probabilities
    probabilities = calculate_probabilities(distances, pheromones)
    print("\nInitial Probabilities:")
    for i, start_node in enumerate(nodes):
        for j, end_node in enumerate(nodes):
            if distances[i, j] > 0:
                print(f"{start_node}-{end_node}: {probabilities[i, j]:.6f}")

    # Run a single ant to demonstrate delta tau calculation
    print("\nRunning a single ant...")
    path, path_length = run_single_ant(nodes, node_indices, distances, pheromones)

    # Convert path indices to node names for display
    path_names = [nodes[i] for i in path]
    print(f"Ant path: {' -> '.join(path_names)}")
    print(f"Path length: {path_length}")

    # Calculate delta tau and tau baru
    results_df = calculate_delta_tau(df, nodes, node_indices, path, path_length, Q)

    # Display results
    print("\nResults after one ant's journey:")
    print(results_df.to_string(index=False))

    # Save results to CSV
    results_df.to_csv("ant_results.csv", index=False)
    print("\nResults saved to ant_results.csv")

if __name__ == "__main__":
    main()

Data loaded successfully:
  Tautan  Bobot  Tau       Eta
0    A-B      4    1  0.250000
1    A-C      3    1  0.333333
2    A-D      6    1  0.166667
3    B-C      2    1  0.500000
4    B-D      5    1  0.200000
5    C-D      1    1  1.000000

Initial Probabilities:
A-B: 0.333333
A-C: 0.444444
A-D: 0.222222
B-A: 0.263158
B-C: 0.526316
B-D: 0.210526
C-A: 0.181818
C-B: 0.272727
C-D: 0.545455
D-A: 0.121951
D-B: 0.146341
D-C: 0.731707

Running a single ant...
Ant path: B -> D -> C -> A -> B
Path length: 13.0

Results after one ant's journey:
Tautan  Bobot  Tau      Eta  Delta Tau  Tau Baru
   A-B      4    1 0.250000   7.692308  8.192308
   A-C      3    1 0.333333   7.692308  8.192308
   A-D      6    1 0.166667   0.000000  0.500000
   B-C      2    1 0.500000   0.000000  0.500000
   B-D      5    1 0.200000   7.692308  8.192308
   C-D      1    1 1.000000   7.692308  8.192308

Results saved to ant_results.csv
